In [68]:
try:
    # Executes if running in Google Colab
    from google.colab import drive
    drive.mount('gdrive/')
    path = "gdrive/My Drive" # Change path to location of data if necessary
except:
    # Executes if running locally (e.g. Anaconda)
    path = "./"



Drive already mounted at gdrive/; to attempt to forcibly remount, call drive.mount("gdrive/", force_remount=True).


In [69]:
#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px

In [70]:
##run this if mounted on google drive  
jsonObj = pd.read_json('/'.join((path,'encounters.jsonl')), lines=True)
df = pd.read_csv('/'.join((path,'patients.csv')))
##if not then run the next two lines which are commented
#path = ""add in your path""
# jsonObj = pd.read_json(path + 'encounters.jsonl', lines=True)
# df = pd.read_csv(path + 'patients.csv')


In [71]:
year=[]
for i in range(len(df)):
  year.append(df['birthdate'][i].split("-")[0])
current_year = []
for i in range(len(df)):
  current_year.append(df['age'][i]+int(year[i]))
df['current_year'] = current_year

In [72]:
df["patient_id"].value_counts()
df_duplicate_pat_drop = df.drop_duplicates(subset = ["patient_id"]).reset_index(drop=True)
df_duplicate_pat_drop["patient_id"].value_counts()


4e8950f7-7766-4b6d-b685-cd28a84000ef    1
2d423123-1aa1-43b5-94a0-969de72af959    1
806a6397-e95b-4f17-b11b-d97a1f354223    1
0c9b61d8-79da-4f25-837a-a14256ef4c78    1
77687178-a9b0-4bf5-a8dc-0e334c02a4e5    1
                                       ..
abbae750-d6a8-4fa9-a3d7-8488d4294be7    1
146a696b-b2a6-4c06-b0b0-688e5d6a905a    1
23d54869-568a-4b10-9d34-adce843f8844    1
faa9340f-a442-4afa-bfd7-a0f4a7255e85    1
e051bc97-039a-47b7-8f7c-29fa078643f3    1
Name: patient_id, Length: 136, dtype: int64

In [73]:
j=0
most_encounter_patient_id = jsonObj["patient_id"].value_counts().index.tolist()[0]
for i in range(len(df_duplicate_pat_drop)):
  if df_duplicate_pat_drop['patient_id'][i] == most_encounter_patient_id:
    if j==0:
      print("The patient with most encounters is: \n {} {}".format(df_duplicate_pat_drop['first_name'][i],df_duplicate_pat_drop['last_name'][i]))
      j+=1

The patient with most encounters is: 
 Rachel Feil


In [74]:
vaccine_df = jsonObj.loc[jsonObj['encounters']=="vaccination"]
vaccine_patients_df =  pd.merge(df_duplicate_pat_drop, vaccine_df, 
                   on='patient_id', 
                   how='inner')


In [75]:
indexes_gender = vaccine_patients_df['gender'].value_counts().index.tolist()
counts_gender = vaccine_patients_df['gender'].value_counts().tolist()
print("The number of vaccine appointnments per gender are:\n {} \t {}\n {}\t {}".format(indexes_gender[0],indexes_gender[1],counts_gender[0],counts_gender[1]))

The number of vaccine appointnments per gender are:
 Male 	 Female
 25	 16


In [76]:
dict_year={'patient_id':df_duplicate_pat_drop['patient_id'],'current_year':df_duplicate_pat_drop['current_year']}
df_year = pd.DataFrame(dict_year)
encounters_df =  pd.merge(jsonObj, df_year,
                   on='patient_id', 
                   how='inner')
indexes_encounters = encounters_df['current_year'].value_counts().index.tolist()
counts_encounters = encounters_df['current_year'].value_counts().tolist()
print("The number of encounters per year are:\n {} \t {}\n {}\t {}".format(indexes_encounters[0],indexes_encounters[1],counts_encounters[0],counts_encounters[1]))

The number of encounters per year are:
 2021 	 2020
 389	 48


***For Non-technical audience:***  


---


**So in the app that i created below, I included 2 drop-down menus. One to select the criteria to be visualized, being the comparison among different age gropus or across different disease types. Second being the criteria you want to distinguish between for the above selected type. i.e distinguish them by age or by encounter type.**    
**I feel this is useful because we can first get a sense of what population we are looking at if we see the graph of age stratified by gender. We can see which disease type is most prevelant across which sex if we look at graph with disease type stratified by gender. We can also observe the different encounters across each group to sense of understand which encounter is common across which age group, if we select age stratified by encounter type. Also can look at which encounter is common across different disease types, if we select disease type stratified by encounter type.**   
**I included the dropdown, to let the visualizers have the liberty to choose what they wish to see, and also not overwhelm them by presenting all information all at once.**  
**The graphs are interactive, meaning you can hover over each bar to get more information**

In [77]:

!pip install jupyter-dash

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# ## Define the app

# # Build App
app = JupyterDash(__name__)



## Import data stored from part1 of this hw
jsonObj = pd.read_json('/'.join((path,'encounters.jsonl')), lines=True)
data = pd.read_csv('/'.join((path,'patients.csv')))
year=[]
for i in range(len(data)):
  year.append(data['birthdate'][i].split("-")[0])
current_year = []
for i in range(len(data)):
  current_year.append(data['age'][i]+int(year[i]))
data['current_year'] = current_year
df_duplicate_pat_drop = data.drop_duplicates(subset = ["patient_id"]).reset_index(drop=True)
dict_year={'patient_id':df_duplicate_pat_drop['patient_id'],'current_year':df_duplicate_pat_drop['current_year']}
df_year = pd.DataFrame(dict_year)
encounters_df =  pd.merge(jsonObj, df_year,
                   on='patient_id', 
                   how='inner')
df = pd.merge(df_duplicate_pat_drop,encounters_df,on="patient_id",how="inner")



## This creates the layout of the page
app.layout = html.Div([
    html.H1("Visualizations for data provided"),
     html.H1(""),
    html.H2("Select from the following dropdown, either age or disease, based on what information you want to visulaize"),
    dcc.Dropdown(options = [
        {'label' : 'age', 'value' : "age"},
        {'label' : 'disease', 'value' : "condition_dataset"}
    ],
    value = "age", id = 'feature_type'
                ),
    html.H2("Select from the following dropdown, either gender or encounters, based on how you wish to look at grouped data"),
    dcc.Dropdown(options = [
        {'label' : 'gender', 'value' : "gender"},
        {'label' : 'encounters', 'value' : "encounters"}
    ],
    value = "gender", id = 'group_by_feature_type'
                ),
    html.H2("Here's a graph, based off your inputs"),
    dcc.Graph(id = 'output-graph')
])

@app.callback(
    Output('output-graph', 'figure'),
    Input(component_id  = 'feature_type', component_property = 'value'),
    Input(component_id  = 'group_by_feature_type', component_property = 'value'))
def get_figure(feature,group_by_feature):
  if group_by_feature=="gender":
    if feature!="age":
      count_series = df.groupby([feature,group_by_feature]).gender.count()
      new_df = count_series.to_frame(name = 'count').reset_index()
    else:
      bins = np.arange(10, 100, 10)

      df['category'] = np.digitize(df.age, bins, right=True)

      counts = df.groupby(['category', group_by_feature]).gender.count()
      new_df = counts.to_frame(name = 'count').reset_index()
      new_df = new_df.rename(columns={"category":"age"}) 
  else:
    if feature!="age":
      count_series = df.groupby([feature,group_by_feature]).encounters.count()
      new_df = count_series.to_frame(name = 'count').reset_index()
    else:
      bins = np.arange(10, 100, 10)

      df['category'] = np.digitize(df.age, bins, right=True)

      counts = df.groupby(['category', group_by_feature]).encounters.count()
      new_df = counts.to_frame(name = 'count').reset_index()
      new_df = new_df.rename(columns={"category":"age"})        
  fig = px.bar(new_df, x=feature, y="count", color=group_by_feature,color_continuous_scale="Bluered", barmode="group")
  if feature=="age":
    fig.update_xaxes(tickangle=45,
                 tickmode = 'array',
                 tickvals = [0,1,2,3,4,5,6,7,8,9],
                 ticktext=["0-10 years","10-20 years","20-30 years","30-40 years","40-50 years","50-60 years","60-70 years","70-80 years","80-90 years","90-100 years"])
  return fig

if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [78]:

!pip install jupyter-dash

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# ## Define the app

# # Build App
app = JupyterDash(__name__)



## Import data stored from part1 of this hw
jsonObj = pd.read_json('/'.join((path,'encounters.jsonl')), lines=True)
data = pd.read_csv('/'.join((path,'patients.csv')))
year=[]
for i in range(len(data)):
  year.append(data['birthdate'][i].split("-")[0])
current_year = []
for i in range(len(data)):
  current_year.append(data['age'][i]+int(year[i]))
data['current_year'] = current_year
df_duplicate_pat_drop = data.drop_duplicates(subset = ["patient_id"]).reset_index(drop=True)
dict_year={'patient_id':df_duplicate_pat_drop['patient_id'],'current_year':df_duplicate_pat_drop['current_year']}
df_year = pd.DataFrame(dict_year)
encounters_df =  pd.merge(jsonObj, df_year,
                   on='patient_id', 
                   how='inner')
df = pd.merge(df_duplicate_pat_drop,encounters_df,on="patient_id",how="inner")



## This creates the layout of the page
app.layout = html.Div([
    html.H1("Visualizations for data provided"),
     html.H1(""),
    html.H2("Select from the following dropdown, either age or disease, based on what information you want to visulaize"),
    dcc.Dropdown(options = [
        {'label' : 'age', 'value' : "age"},
        {'label' : 'disease', 'value' : "condition_dataset"}
    ],
    value = "age", id = 'feature_type'
                ),
    html.H2("Select from the following dropdown, either gender or encounters, based on how you wish to look at grouped data"),
    dcc.Dropdown(options = [
        {'label' : 'gender', 'value' : "gender"},
        {'label' : 'encounters', 'value' : "encounters"}
    ],
    value = "gender", id = 'group_by_feature_type'
                ),
    html.H2("Here's a graph, based off your inputs"),
    dcc.Graph(id = 'output-graph')
])

@app.callback(
    Output('output-graph', 'figure'),
    Input(component_id  = 'feature_type', component_property = 'value'),
    Input(component_id  = 'group_by_feature_type', component_property = 'value'))
def get_figure(feature,group_by_feature):
  if group_by_feature=="gender":
    if feature!="age":
      count_series = df.groupby([feature,group_by_feature]).gender.count()
      new_df = count_series.to_frame(name = 'count').reset_index()
    else:
      bins = np.arange(10, 100, 10)

      df['category'] = np.digitize(df.age, bins, right=True)

      counts = df.groupby(['category', group_by_feature]).gender.count()
      new_df = counts.to_frame(name = 'count').reset_index()
      new_df = new_df.rename(columns={"category":"age"}) 
  else:
    if feature!="age":
      count_series = df.groupby([feature,group_by_feature]).encounters.count()
      new_df = count_series.to_frame(name = 'count').reset_index()
    else:
      bins = np.arange(10, 100, 10)

      df['category'] = np.digitize(df.age, bins, right=True)

      counts = df.groupby(['category', group_by_feature]).encounters.count()
      new_df = counts.to_frame(name = 'count').reset_index()
      new_df = new_df.rename(columns={"category":"age"})        
  fig = px.bar(new_df, x=feature, y="count", color=group_by_feature,color_continuous_scale="Bluered", barmode="group")
  if feature=="age":
    fig.update_xaxes(tickangle=45,
                 tickmode = 'array',
                 tickvals = [0,1,2,3,4,5,6,7,8,9],
                 ticktext=["0-10 years","10-20 years","20-30 years","30-40 years","40-50 years","50-60 years","60-70 years","70-80 years","80-90 years","90-100 years"])
  return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>